In [1]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from geopy.distance import geodesic
from openrouteservice import Client
import openrouteservice.convert as convert
import time
from shapely.geometry import Point

# Descargar los POIs de OSMnx

In [ ]:
# Definir la ubicación de Valencia en osmnx
valencia_location = "Valencia, Spain"

# Descargar los barrios de Valencia [geometrías administrativas (admin_level=10)]
valencia_boundaries = ox.features.features_from_address(valencia_location, tags={"admin_level": "10"}, dist = 12000)
# Guardar los barrios en un geojson
valencia_boundaries.to_file("Barrios_valencia.geojson", driver="GeoJSON")

In [ ]:
# Descargar datos de colegios, hospitales, parques, paradas de bus y tram
schools = ox.features.features_from_address(valencia_location, tags={'amenity': 'school'}, dist=9000)
schools['geometry'] = schools['geometry'].apply(lambda geom: geom.centroid if geom.geom_type == 'Polygon' else geom) # Algunos POI están guardados como poligosnos por lo que guardamos el centro de estos poligonos

hospitals = ox.features.features_from_address(valencia_location, tags={'amenity': 'hospital'}, dist=9000)

parques = ox.features.features_from_address(valencia_location, tags={"leisure": "park"}, dist=9000)
parques['geometry'] = parques['geometry'].apply(lambda geom: geom.centroid if geom.geom_type == 'Polygon' else geom)
# Reemplazar los nombres NaN con "Zona Verde"
parques['name'] = parques['name'].fillna('Zona Verde') # Esto se hace porque algunos parques no tenían nombre

bus_stops = ox.features_from_address(valencia_location, tags={"highway": "bus_stop"}, dist=9000)

tram_stops = ox.features_from_address(valencia_location, tags={"railway": "tram_stop"}, dist=9000)

In [ ]:
# Filtrar colegios, hospitales, parques, paradas de bus y tram con coordenadas (latitud, longitud). Así nos quedamos solo con los nodos
school_locations = schools[schools.geometry.type == 'Point']
hospital_locations = hospitals[hospitals.geometry.type == 'Point']
bus_locations = bus_stops[bus_stops.geometry.type == 'Point']
tram_locations = tram_stops[tram_stops.geometry.type == 'Point']

# Extraer las coordenadas de los colegios, hospitales, parques, paradas de bus y tram
school_coords = [(point.y, point.x) for point in school_locations.geometry]
hospital_coords = [(point.y, point.x) for point in hospital_locations.geometry]
bus_coords = [(point.y, point.x) for point in bus_locations.geometry]
tram_coords = [(point.y, point.x) for point in tram_locations.geometry]

# Extraer las coordenadas y nombres de los colegios, hospitales, parques, paradas de bus y tram
school_coords_names = [(point.y, point.x, name) for point, name in zip(school_locations.geometry, school_locations['name'])]
hospital_coords_names = [(point.y, point.x, name) for point, name in zip(hospital_locations.geometry, hospital_locations['name'])]
bus_coords_names = [(point.y, point.x, name) for point, name in zip(bus_locations.geometry, bus_locations['name'])]
tram_coords_names = [(point.y, point.x, name) for point, name in zip(tram_locations.geometry, tram_locations['name'])]

# Como los parques están definidos en su mayoria como poligonos hay que realizar algunas transformaciones adicionales:
# Obtener los centroides de los polígonos
parques['centroid'] = parques.centroid

# Extraer las coordenadas de los centroides
parques['coords'] = parques['centroid'].apply(lambda x: (x.y, x.x))

# Crear una lista de tuples con nombre y coordenadas de los parques
parques_coords_names = [(row['centroid'].y, row['centroid'].x, row['name'] if 'name' in row else 'Unnamed Park') for idx, row in parques.iterrows()]

# Funciones para calcular rutas y distancias

In [ ]:
# Calcular la distancia al POI más cercano para cada ubicación dada (lat, lon)
def nearest_distance_and_name(lat, lon, locations):
    min_distance = float('inf')
    nearest_name = ''
    nearest_coords = None
    for location in locations:
        distance = geodesic((lat, lon), (location[0], location[1])).meters
        if distance < min_distance:
            min_distance = distance
            nearest_name = location[2]  # El nombre del lugar está en la tercera posición
            nearest_coords = (location[0], location[1])
    return min_distance, nearest_name, nearest_coords

In [ ]:
# Inicializar el cliente de OpenRouteService
ors_client = Client(key='5b3ce3597851110001cf624859af420fdd51467a9d1b5f39737c6113')

# Función para obtener la ruta más rápida en coche
def get_route(coords, profile='driving-car'):
    try:
        response = ors_client.directions(
            coordinates=coords,
            profile = profile,
            format='geojson'
        )
        if 'features' in response:
            # Acceso a las coordenadas directamente
            geometry = response['features'][0]['geometry']
            coordinates = geometry['coordinates']
            duration = response['features'][0]['properties']['segments'][0]['duration']


            # Verifica que las coordenadas sean una lista de puntos válidos
            if isinstance(coordinates, list) and all(isinstance(coord, list) and len(coord) == 2 for coord in coordinates):
                route_coords = [(point[1], point[0]) for point in coordinates]  # Intercambio de latitud y longitud
                return route_coords, duration
            else:
                print("Invalid coordinates format.")
                return [], 0
        else:
            print("No features found in the response.")
            return [], 0
    except Exception as e:
        print("Error fetching route:", e)
        return [], 0

# Modificación de los datasets de pisos y POIs

In [ ]:
# Parimos del Dataset de Idealista
idealista18 = pd.read_csv('idealista18.csv')
# Separar el dataset de idealista18 en 3 datasets según la ciudad:
madrid = idealista18[idealista18['CITYNAME']=='Madrid']
barcelona = idealista18[idealista18['CITYNAME']=='Barcelona']
valencia = idealista18[idealista18['CITYNAME']=='Valencia']
# Ordenar el dataset de Valencia:
valencia = valencia.sort_values('DISTANCE_TO_CITY_CENTER')
# Definir las distancias a los puntos de interés más cercanos para cada piso:
valencia['distancia_al_colegio_mas_cercano'], valencia['colegio_mas_cercano'], valencia['colegio_coords'] = zip(*valencia.apply(
    lambda row: nearest_distance_and_name(row['LATITUDE'], row['LONGITUDE'], school_coords_names), axis=1))
valencia['distancia_al_hospital_mas_cercano'], valencia['hospital_mas_cercano'], valencia['hospital_coords'] = zip(*valencia.apply(
    lambda row: nearest_distance_and_name(row['LATITUDE'], row['LONGITUDE'], hospital_coords_names), axis=1))
valencia['distancia_al_parque_mas_cercano'], valencia['parque_mas_cercano'], valencia['parque_coords'] = zip(*valencia.apply(
    lambda row: nearest_distance_and_name(row['LATITUDE'], row['LONGITUDE'], parques_coords_names), axis=1))
valencia['distancia_al_bus_mas_cercano'], valencia['bus_mas_cercano'], valencia['bus_coords'] = zip(*valencia.apply(
    lambda row: nearest_distance_and_name(row['LATITUDE'], row['LONGITUDE'], bus_coords_names), axis=1))
valencia['distancia_al_tram_mas_cercano'], valencia['tram_mas_cercano'], valencia['tram_coords'] = zip(*valencia.apply(
    lambda row: nearest_distance_and_name(row['LATITUDE'], row['LONGITUDE'], tram_coords_names), axis=1))
# Separamos las coordenadas de los puntos de interés en latitud y longitud
valencia['colegio_lat'], valencia['colegio_lon'] = zip(*valencia['colegio_coords'].dropna().apply(lambda x: x if len(x) == 2 else (None, None)))
valencia['hospital_lat'], valencia['hospital_lon'] = zip(*valencia['hospital_coords'].dropna().apply(lambda x: x if len(x) == 2 else (None, None)))
valencia['parque_lat'], valencia['parque_lon'] = zip(*valencia['parque_coords'].dropna().apply(lambda x: x if len(x) == 2 else (None, None)))
valencia['bus_lat'], valencia['bus_lon'] = zip(*valencia['bus_coords'].dropna().apply(lambda x: x if len(x) == 2 else (None, None)))
valencia['tram_lat'], valencia['tram_lon'] = zip(*valencia['tram_coords'].dropna().apply(lambda x: x if len(x) == 2 else (None, None)))
# Guardamos el dataset de Valencia:
valencia.to_csv('valencia.csv', index = False)

In [ ]:
# Convertir el DataFrame de pisos en un GeoDataFrame
valencia_gdf = gpd.GeoDataFrame(valencia, geometry=gpd.points_from_xy(valencia['LONGITUDE'], valencia['LATITUDE']), crs="EPSG:4326")

# Realizar la unión espacial para asignar cada piso a un barrio
pisos_con_barrios = gpd.sjoin(valencia_gdf, valencia_neighborhoods, how="left")#, op="within")
schools_con_barrios = gpd.sjoin(schools, valencia_neighborhoods, how="left")#, predicate="within")
hospitals_con_barrios = gpd.sjoin(hospitals, valencia_neighborhoods, how="left")#, predicate="within")
parques_con_barrios = gpd.sjoin(parques, valencia_neighborhoods, how="left")#, predicate="within")
bus_con_barrios = gpd.sjoin(bus_stops, valencia_neighborhoods, how="left")#, predicate="within")
tram_con_barrios = gpd.sjoin(tram_stops, valencia_neighborhoods, how="left")#, predicate="within")

# El resultado incluirá una nueva columna con el nombre del barrio
pisos_con_barrios = pisos_con_barrios.rename(columns={"name": "Barrio"})
schools_con_barrios = schools_con_barrios.rename(columns={"name_right": "Barrio"})
hospitals_con_barrios = hospitals_con_barrios.rename(columns={"name_right": "Barrio"})
parques_con_barrios = parques_con_barrios.rename(columns={"name_right": "Barrio"})
bus_con_barrios = bus_con_barrios.rename(columns={"name_right": "Barrio"})
tram_con_barrios = tram_con_barrios.rename(columns={"name_right": "Barrio"})

# Guardar los datasets 
pisos_con_barrios.to_csv('pisos_con_barrios.csv', index=False)
schools_con_barrios.to_csv('colegios_con_barrios.csv', index = False)
hospitals_con_barrios.to_csv('hospitales_con_barrios.csv', index=False)
parques_con_barrios.to_csv('parques_con_barrios.csv', index=False)
bus_con_barrios.to_csv('bus_con_barrios.csv', index=False)
tram_con_barrios.to_csv('tram_con_barrios.csv', index=False)

# Guardar RUTAS para no saturar la API

In [ ]:
conteo_pisos = pisos_con_barrio['Barrio'].value_counts().reset_index()
conteo_pisos.columns = ['Barrio', 'cantidad']
barrios = conteo_pisos['Barrio'].unique()
for _, row in conteo_pisos.iterrows():
    print(row)

In [ ]:
# Hemos guardado las rutas de los pisos para diferentes barrios
numero = 9 # Iterar sobre todos los barrios
barrio = barrios[numero]
pisos_barrio_numero = pisos_con_barrio[pisos_con_barrio['Barrio']==barrio]

rutas_colegios_car = []
tiempos_colegios_car = []
rutas_parques_car = []
tiempos_parques_car = []
rutas_hospitales_car = []
tiempos_hospitales_car = []

rutas_colegios_andando = []
tiempos_colegios_andando = []
rutas_parques_andando = []
tiempos_parques_andando = []
rutas_hospitales_andando = []
tiempos_hospitales_andando = []

for _, row in pisos_barrio_numero.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    colegio_lat = row['colegio_lat']
    colegio_lon = row['colegio_lon']
    hospital_lat = row['hospital_lat']
    hospital_lon = row['hospital_lon']
    parque_lat = row['parque_lat']
    parque_lon = row['parque_lon']

    # Obtener la ruta al colegio más cercano
    # En coche
    school_route_car, school_time_car = get_route([(lon, lat), (colegio_lon, colegio_lat)], profile='driving-car')
    time.sleep(5)  # Esperar 1 segundo antes de la siguiente petición
    # A pie
    school_route_andando, school_time_andando = get_route([(lon, lat), (colegio_lon, colegio_lat)], profile='foot-walking')
    time.sleep(5)  # Esperar 1 segundo antes de la siguiente petición

    # Obtener la ruta al hospital más cercano
    # En coche
    hospital_route_car, hospital_time_car = get_route([(lon, lat), (hospital_lon, hospital_lat)], profile='driving-car')
    time.sleep(5)  # Esperar 1 segundo antes de la siguiente petición
    # A pie
    hospital_route_andando, hospital_time_andando = get_route([(lon, lat), (hospital_lon, hospital_lat)], profile='foot-walking')
    time.sleep(5)  # Esperar 1 segundo antes de la siguiente petición

    # Obtener la ruta al hospital más cercano
    # En coche
    park_route_car, park_time_car = get_route([(lon, lat), (parque_lon, parque_lat)], profile='driving-car')
    time.sleep(5)  # Esperar 1 segundo antes de la siguiente petición
    # A pie
    park_route_andando, park_time_andando = get_route([(lon, lat), (parque_lon, parque_lat)], profile='foot-walking')
    time.sleep(5)  # Esperar 1 segundo antes de la siguiente petición

    rutas_colegios_car.append(school_route_car)
    rutas_parques_car.append(park_route_car)
    rutas_hospitales_car.append(hospital_route_car)

    tiempos_colegios_car.append(school_time_car)
    tiempos_parques_car.append(park_time_car)
    tiempos_hospitales_car.append(hospital_time_car)

    rutas_colegios_andando.append(school_route_andando)
    rutas_parques_andando.append(park_route_andando)
    rutas_hospitales_andando.append(hospital_route_andando)

    tiempos_colegios_andando.append(school_time_andando)
    tiempos_parques_andando.append(park_time_andando)
    tiempos_hospitales_andando.append(hospital_time_andando)

pisos_barrio_numero['tiempo_colegio_coche'] = tiempos_colegios_car
pisos_barrio_numero['tiempo_hospital_coche'] = tiempos_hospitales_car
pisos_barrio_numero['tiempo_parque_coche'] = tiempos_parques_car

pisos_barrio_numero['ruta_colegio_coche'] = rutas_colegios_car
pisos_barrio_numero['ruta_hospital_coche'] = rutas_hospitales_car
pisos_barrio_numero['ruta_parque_coche'] = rutas_parques_car

pisos_barrio_numero['tiempo_colegio_andando'] = tiempos_colegios_andando
pisos_barrio_numero['tiempo_hospital_andando'] = tiempos_hospitales_andando
pisos_barrio_numero['tiempo_parque_andando'] = tiempos_parques_andando

pisos_barrio_numero['ruta_colegio_andando'] = rutas_colegios_andando
pisos_barrio_numero['ruta_hospital_andando'] = rutas_hospitales_andando
pisos_barrio_numero['ruta_parque_andando'] = rutas_parques_andando

pisos_barrio_numero.to_json(f'{barrio}.json')

In [ ]:
# Barrios de los que hemos guardado las rutas
barrios_disponibles = ['Albors', 'Beniferri', 'Camí Real', 'Ciutat Fallera', 'Ciutat Jardí', 'el Cabanyal - el Canyamelar', 
                      'Exposició', 'Faitanar', 'Jaume Roig', 'la Carrasca', 'la Creu Coberta', 'la Seu', 'Patraix', 'Poble Nou',
                      'Safranar', 'Sant Francesc', 'Trinitat', 'Vara de Quart', 'la Petxina']
df = pd.read_json(f'{barrios_disponibles[0]}.json')
df = df.drop(['distancia_al_hospital_mas_cercano.1'], axis = 1)
for barrio in barrios_disponibles[1::]:
    new_barrio = pd.read_json(f'{barrio}.json')
    new_barrio = new_barrio[df.columns]
    df = pd.concat([df, new_barrio])


pisos_sin_rutas = pd.read_csv('pisos_con_barrio.csv')
pisos_sin_rutas = pisos_sin_rutas.drop(['distancia_al_hospital_mas_cercano.1'], axis = 1)

# Filtrar los pisos que NO pertenecen a los barrios con rutas
pisos_sin_rutas = pisos_sin_rutas[~pisos_sin_rutas['Barrio'].isin(barrios_disponibles)]

# Concatenar ambos dataframes
pisos = pd.concat([df, pisos_sin_rutas], ignore_index=True)

# Guardamos el Dataset FINAL de los pisos, el cual tiene las rutas de los pisos de los barrios que hemos calculado
pisos = pd.read_json('DATASET_FINAL.json')